# Collect all data

In [1]:
import struct

def get_byte_list(lbl_file_name, img_file_name):
    '''
    Returns a list of tuples,
    each tuple contains a label and an image, both in bytes.
    '''

    tuples = []
    with open(lbl_file_name, 'rb') as lbl_file, open(img_file_name, 'rb') as img_file:

        magic_number, num = struct.unpack('>II', lbl_file.read(8))
        _magic, _num, rows, cols = struct.unpack('>IIII', img_file.read(16))

        assert(num == _num)

        for i in range(num):
            label = lbl_file.read(1)
            img = img_file.read(rows*cols)

            tuples.append((label, img))

    return tuples

In [2]:
!ls -l ../src/data/mnist/t*

-rw-rw-r-- 1 adrian adrian  7840016 maj 14 16:08 ../src/data/mnist/t10k-images.idx3-ubyte
-rw-rw-r-- 1 adrian adrian    10008 maj 14 16:08 ../src/data/mnist/t10k-labels.idx1-ubyte
-rw-rw-r-- 1 adrian adrian 47040016 maj 14 16:08 ../src/data/mnist/train-images.idx3-ubyte
-rw-rw-r-- 1 adrian adrian    60008 maj 14 16:08 ../src/data/mnist/train-labels.idx1-ubyte


In [3]:
from pathlib import Path

DATA_DIR = Path('../src/data/mnist')
train_data = get_byte_list(DATA_DIR / 'train-labels.idx1-ubyte', DATA_DIR / 'train-images.idx3-ubyte')
test_data = get_byte_list(DATA_DIR / 't10k-labels.idx1-ubyte', DATA_DIR / 't10k-images.idx3-ubyte')
data = train_data + test_data
print('tot num data', len(data))

tot num data 70000


# Shuffle the data

In [4]:
SEED = 20180502

In [5]:
from random import shuffle
from random import seed

seed(SEED)
print('first 4 before:', list(zip(*data[:4]))[0])
shuffle(data)
print('first 4 after:', list(zip(*data[:4]))[0])

first 4 before: (b'\x05', b'\x00', b'\x04', b'\x01')
first 4 after: (b'\x01', b'\x06', b'\x02', b'\x01')


# Split into folds

In [6]:
def print_folds(folds):
    for i in range(len(folds[0])):
        print(i, end='\t')
        for fold in folds:
            print(int.from_bytes(fold[i][0], byteorder='big'), end='\t')
        print()

In [7]:
NUM_FOLDS = 5

n_data = len(data)
n_data_per_fold = n_data // NUM_FOLDS
print(n_data_per_fold, 'per fold; rest:', n_data%NUM_FOLDS)

tmp_data = data
folds = []
for i in range(NUM_FOLDS):
    heads = tmp_data[:n_data_per_fold]
    tail = tmp_data[n_data_per_fold:]
    
    folds.append(heads)
    tmp_data = tail

14000 per fold; rest: 0


In [8]:
print_folds(folds)

0	1	0	8	0	5	
1	6	3	6	4	9	
2	2	7	9	8	8	
3	1	5	5	3	6	
4	5	9	1	6	0	
5	5	3	3	1	8	
6	6	3	3	4	1	
7	9	2	6	8	1	
8	8	9	1	3	9	
9	0	9	5	6	8	
10	0	8	9	6	2	
11	8	7	4	3	7	
12	0	6	6	0	8	
13	3	8	5	4	5	
14	4	8	7	3	0	
15	9	9	3	3	1	
16	4	3	0	5	2	
17	8	2	7	3	0	
18	1	8	1	1	4	
19	3	6	1	4	7	
20	3	4	0	9	0	
21	4	1	0	4	2	
22	0	1	2	1	8	
23	4	6	7	0	0	
24	8	3	7	3	8	
25	5	9	4	1	1	
26	3	0	3	8	2	
27	4	5	0	1	0	
28	7	7	7	6	1	
29	4	9	6	3	3	
30	1	4	6	9	7	
31	3	3	3	2	6	
32	4	7	0	8	0	
33	5	5	8	9	5	
34	9	2	0	0	4	
35	7	7	4	2	9	
36	0	1	2	2	3	
37	5	9	4	4	1	
38	6	0	9	2	4	
39	1	8	7	0	8	
40	8	1	2	9	0	
41	7	0	8	4	9	
42	8	2	5	7	7	
43	9	3	6	9	5	
44	7	2	9	3	9	
45	2	8	2	6	1	
46	8	2	7	6	0	
47	0	7	3	6	6	
48	5	0	0	8	0	
49	1	7	8	8	4	
50	6	0	2	7	9	
51	1	3	2	7	8	
52	7	4	5	2	9	
53	8	0	4	7	8	
54	2	7	8	1	6	
55	6	2	9	1	6	
56	2	6	8	4	5	
57	1	3	4	4	2	
58	0	6	1	1	4	
59	7	7	8	2	4	
60	8	5	1	1	0	
61	1	4	8	6	3	
62	7	9	8	9	3	
63	9	5	9	9	6	
64	6	9	2	6	4	
65	2	6	0	5	1	
66	5	3	7	1	5	
67	8	9	8	9	0	
68	4	6	1	6	4	
69	9	1	7	1	3	
70	7	9	9	8	0	
71	8	8	3	1	7	
72

560	1	1	0	6	7	
561	4	5	9	3	1	
562	8	0	6	2	3	
563	3	8	1	2	9	
564	9	8	6	1	4	
565	2	1	7	5	9	
566	8	6	5	4	9	
567	2	4	8	4	8	
568	0	4	2	3	5	
569	7	1	3	9	7	
570	7	1	1	3	3	
571	0	5	8	3	7	
572	6	7	6	6	2	
573	6	3	9	5	3	
574	4	9	0	4	9	
575	2	2	4	1	1	
576	5	8	3	4	8	
577	0	4	2	8	0	
578	1	0	2	3	9	
579	2	6	6	4	7	
580	6	6	8	1	6	
581	8	7	3	3	8	
582	7	4	7	1	3	
583	7	3	0	6	2	
584	3	8	4	2	9	
585	0	5	3	1	7	
586	9	0	9	4	6	
587	4	5	5	8	6	
588	0	6	7	9	5	
589	3	9	9	5	5	
590	1	1	0	4	4	
591	1	9	5	5	2	
592	0	6	8	9	2	
593	0	8	5	0	7	
594	3	7	9	6	0	
595	2	9	1	3	0	
596	1	8	4	7	5	
597	3	5	8	4	8	
598	6	4	7	0	7	
599	6	8	4	3	9	
600	6	9	9	5	5	
601	1	7	6	6	0	
602	7	7	7	2	0	
603	6	2	3	6	1	
604	9	7	1	5	1	
605	9	3	9	6	2	
606	7	3	0	7	1	
607	6	8	1	5	4	
608	8	4	0	9	5	
609	1	6	5	6	9	
610	5	3	6	1	9	
611	2	3	7	0	9	
612	0	0	8	1	6	
613	2	0	4	2	9	
614	3	5	2	1	2	
615	4	9	6	8	9	
616	2	7	0	3	5	
617	8	3	4	2	4	
618	1	0	9	4	5	
619	9	6	2	2	2	
620	3	7	4	0	3	
621	5	0	2	1	1	
622	3	5	1	6	6	
623	5	9	4	1	7	
624	4	0	4	0	0	
625	3	8	8	7	1	
626	4	3	4	

1219	1	3	8	9	0	
1220	5	2	4	2	2	
1221	4	6	4	6	0	
1222	0	5	2	0	9	
1223	9	2	2	1	9	
1224	6	2	4	3	9	
1225	6	2	0	3	1	
1226	3	0	8	4	4	
1227	5	3	2	5	0	
1228	5	9	9	0	4	
1229	3	4	1	7	1	
1230	6	6	1	0	9	
1231	7	5	0	4	4	
1232	1	5	2	1	5	
1233	0	5	7	7	4	
1234	5	3	0	7	0	
1235	0	3	7	8	0	
1236	5	2	3	4	3	
1237	8	4	6	9	1	
1238	6	8	7	5	9	
1239	8	8	3	9	6	
1240	8	9	0	7	2	
1241	3	8	3	5	2	
1242	6	8	1	3	9	
1243	0	8	9	6	0	
1244	2	2	6	8	4	
1245	2	1	6	5	6	
1246	1	0	3	6	1	
1247	5	6	0	7	1	
1248	3	0	6	5	0	
1249	5	5	5	9	6	
1250	4	3	3	6	7	
1251	1	4	2	0	7	
1252	7	7	6	8	7	
1253	9	2	3	7	7	
1254	7	0	1	2	3	
1255	3	3	3	2	8	
1256	4	8	7	0	9	
1257	1	3	1	3	9	
1258	1	6	9	4	4	
1259	2	9	8	6	8	
1260	1	7	3	4	8	
1261	6	0	6	4	3	
1262	5	3	6	6	9	
1263	4	7	9	8	2	
1264	7	3	6	6	3	
1265	6	0	8	3	6	
1266	7	5	5	1	9	
1267	2	9	0	3	3	
1268	1	8	9	3	6	
1269	0	4	2	3	6	
1270	5	1	3	6	0	
1271	1	8	1	9	4	
1272	4	7	3	9	1	
1273	7	8	7	3	2	
1274	1	9	5	4	7	
1275	1	1	5	0	2	
1276	9	4	6	7	7	
1277	5	9	3	4	8	
1278	1	6	0	3	8	
1279	5	2	2	6	5	
1280	2	1	2	1	1	
1281	0	8

1756	3	2	9	7	6	
1757	0	3	8	3	0	
1758	1	1	5	7	1	
1759	6	7	4	5	6	
1760	2	6	7	9	9	
1761	4	4	4	3	7	
1762	4	4	1	1	5	
1763	2	6	8	1	2	
1764	5	2	6	3	5	
1765	8	4	5	3	1	
1766	3	4	7	9	7	
1767	6	8	7	9	8	
1768	4	1	5	0	2	
1769	1	7	8	7	4	
1770	2	0	1	0	7	
1771	3	2	9	7	9	
1772	8	7	8	7	4	
1773	1	7	3	6	0	
1774	2	7	2	0	7	
1775	4	7	0	7	8	
1776	9	3	2	2	3	
1777	3	3	9	0	4	
1778	8	8	2	5	2	
1779	8	1	8	2	4	
1780	3	8	9	6	9	
1781	4	5	7	9	3	
1782	1	6	6	8	4	
1783	5	8	2	1	5	
1784	5	1	0	4	7	
1785	2	2	5	2	9	
1786	6	2	4	7	7	
1787	2	6	8	5	2	
1788	7	6	0	8	0	
1789	0	2	9	6	9	
1790	2	9	3	1	9	
1791	2	0	0	0	5	
1792	9	0	4	5	0	
1793	5	8	9	2	1	
1794	3	4	6	8	6	
1795	4	8	2	2	7	
1796	2	8	2	9	7	
1797	7	3	4	4	0	
1798	5	9	1	3	6	
1799	5	1	5	8	0	
1800	9	3	8	9	5	
1801	9	4	3	7	9	
1802	8	6	6	4	0	
1803	4	2	5	9	2	
1804	0	1	2	1	7	
1805	1	5	2	6	9	
1806	1	1	9	0	5	
1807	9	1	9	7	6	
1808	7	5	8	4	4	
1809	4	9	6	2	7	
1810	4	9	0	8	7	
1811	5	1	3	0	2	
1812	5	1	7	3	8	
1813	8	5	7	4	2	
1814	5	7	3	4	2	
1815	8	7	7	5	0	
1816	3	8	1	7	3	
1817	9	4	8	6	0	
1818	3	5

2294	4	8	6	3	2	
2295	3	7	0	2	9	
2296	4	0	4	8	9	
2297	0	6	2	0	0	
2298	7	9	0	0	6	
2299	0	7	0	4	1	
2300	7	6	8	1	9	
2301	2	8	7	0	6	
2302	1	4	9	2	3	
2303	9	0	9	5	3	
2304	0	6	4	9	1	
2305	1	0	1	0	8	
2306	3	8	7	3	5	
2307	7	6	9	4	9	
2308	2	6	2	6	4	
2309	4	8	5	6	9	
2310	5	3	4	3	9	
2311	2	3	2	2	8	
2312	0	6	7	5	3	
2313	6	3	5	7	5	
2314	2	0	5	7	2	
2315	6	8	9	7	3	
2316	5	1	2	0	2	
2317	7	7	2	1	2	
2318	2	3	8	9	5	
2319	2	1	9	7	2	
2320	4	4	3	1	2	
2321	0	5	1	7	8	
2322	1	2	2	2	8	
2323	1	3	1	9	5	
2324	6	6	1	5	5	
2325	5	2	4	7	0	
2326	1	7	0	3	0	
2327	3	9	1	9	4	
2328	8	2	1	5	8	
2329	1	3	0	3	9	
2330	9	2	2	8	2	
2331	3	1	5	5	6	
2332	0	4	5	4	0	
2333	3	3	3	6	7	
2334	9	0	9	4	2	
2335	2	2	0	4	6	
2336	9	2	1	8	1	
2337	5	4	0	3	1	
2338	2	8	9	3	9	
2339	7	4	5	1	1	
2340	1	7	0	8	8	
2341	8	1	2	2	7	
2342	1	4	0	2	9	
2343	6	3	9	4	3	
2344	3	0	1	5	0	
2345	7	2	3	8	8	
2346	2	0	5	6	8	
2347	4	4	8	8	6	
2348	8	3	9	9	0	
2349	5	8	4	2	4	
2350	5	1	2	8	8	
2351	6	6	3	2	4	
2352	8	8	9	6	4	
2353	8	6	3	1	4	
2354	5	5	6	5	1	
2355	6	5	9	0	3	
2356	4	2

...

10617	1	3	9	6	5	
10618	3	6	6	5	5	
10619	1	6	5	9	6	
10620	0	0	5	3	0	
10621	1	6	0	4	6	
10622	8	5	4	8	5	
10623	1	2	6	7	2	
10624	5	9	8	7	3	
10625	5	2	2	4	9	
10626	0	4	6	0	9	
10627	9	5	6	9	2	
10628	5	2	5	2	2	
10629	7	0	7	7	3	
10630	0	9	0	6	0	
10631	1	4	2	8	2	
10632	8	0	4	1	4	
10633	8	6	1	5	9	
10634	9	0	7	8	2	
10635	0	5	0	6	9	
10636	9	2	7	3	4	
10637	5	1	0	1	6	
10638	9	7	3	2	8	
10639	5	3	8	4	4	
10640	2	7	7	0	8	
10641	5	3	3	0	8	
10642	2	7	8	1	3	
10643	4	1	0	6	7	
10644	8	9	3	7	8	
10645	1	7	0	8	0	
10646	4	1	3	3	8	
10647	2	6	5	3	6	
10648	6	0	3	1	8	
10649	5	6	3	4	4	
10650	7	3	1	2	9	
10651	7	4	9	8	9	
10652	9	8	0	9	0	
10653	4	7	2	5	4	
10654	7	7	7	5	6	
10655	7	7	2	3	9	
10656	9	0	6	8	7	
10657	0	8	5	3	9	
10658	1	0	9	0	4	
10659	2	7	2	9	5	
10660	3	6	5	8	9	
10661	5	5	6	7	1	
10662	6	4	4	2	2	
10663	8	8	5	8	7	
10664	4	3	2	7	2	
10665	5	9	5	3	5	
10666	2	5	5	9	0	
10667	4	9	6	8	1	
10668	9	3	8	3	3	
10669	8	2	9	7	8	
10670	2	8	8	7	1	
10671	1	2	6	6	2	
10672	9	7	8	7	9	
10673	6	6	9	9	3	
10674	3	1	8	6	4	
10675	8	9	4	3	

11248	8	4	0	3	6	
11249	0	1	3	3	7	
11250	5	3	9	3	6	
11251	2	4	9	3	7	
11252	0	4	5	6	2	
11253	0	8	6	5	5	
11254	0	2	5	7	4	
11255	3	7	8	1	4	
11256	3	9	8	4	2	
11257	5	3	3	1	7	
11258	9	7	6	7	1	
11259	1	4	1	4	3	
11260	3	1	3	6	3	
11261	5	8	6	9	6	
11262	8	8	3	6	8	
11263	0	1	9	2	2	
11264	5	7	2	0	3	
11265	6	1	4	7	4	
11266	3	1	7	8	8	
11267	4	3	1	8	8	
11268	0	1	4	3	1	
11269	7	0	1	5	7	
11270	2	2	2	1	5	
11271	3	2	7	8	4	
11272	3	9	1	4	1	
11273	7	5	6	5	4	
11274	5	6	6	0	4	
11275	1	9	3	4	8	
11276	7	0	2	3	7	
11277	0	9	4	8	9	
11278	3	5	0	4	7	
11279	0	1	1	0	0	
11280	0	3	8	6	3	
11281	2	8	1	7	4	
11282	8	0	6	7	7	
11283	6	4	0	2	9	
11284	7	1	9	6	1	
11285	7	8	7	2	2	
11286	6	6	4	5	9	
11287	5	8	0	7	7	
11288	3	4	2	8	0	
11289	6	6	0	4	9	
11290	1	9	0	9	4	
11291	3	0	4	2	0	
11292	7	1	4	0	2	
11293	2	7	6	6	8	
11294	1	7	8	9	2	
11295	0	7	1	2	7	
11296	7	2	4	0	5	
11297	9	5	0	9	9	
11298	5	3	0	4	5	
11299	4	2	3	9	0	
11300	2	4	0	9	1	
11301	5	5	6	2	3	
11302	0	9	8	3	1	
11303	1	9	4	3	6	
11304	2	9	0	8	8	
11305	9	5	3	7	0	
11306	5	9	7	5	

12038	3	9	1	3	0	
12039	5	4	3	4	5	
12040	8	7	0	3	7	
12041	1	2	1	5	1	
12042	1	6	3	8	5	
12043	8	6	2	1	4	
12044	9	0	4	8	4	
12045	3	5	3	2	7	
12046	0	3	5	7	0	
12047	4	3	1	8	1	
12048	2	5	3	3	1	
12049	8	9	7	0	4	
12050	9	3	1	3	3	
12051	9	0	4	6	4	
12052	3	7	4	8	7	
12053	5	5	8	8	9	
12054	6	7	3	0	2	
12055	7	1	5	0	2	
12056	7	3	0	4	2	
12057	7	1	9	7	1	
12058	4	8	9	2	4	
12059	5	2	3	5	3	
12060	1	8	4	6	3	
12061	4	9	9	0	5	
12062	5	1	7	6	2	
12063	1	3	7	1	0	
12064	4	4	2	3	1	
12065	3	6	2	2	5	
12066	6	3	2	9	9	
12067	4	8	0	7	6	
12068	9	2	8	1	1	
12069	9	0	6	2	9	
12070	2	6	3	7	8	
12071	3	7	8	8	7	
12072	1	3	3	0	6	
12073	7	1	8	5	0	
12074	4	2	1	2	9	
12075	0	6	9	0	8	
12076	3	1	5	9	6	
12077	7	0	1	4	0	
12078	6	4	7	8	7	
12079	4	8	4	2	4	
12080	3	0	2	2	6	
12081	0	5	8	7	0	
12082	3	0	2	3	6	
12083	4	2	4	3	5	
12084	4	8	8	0	3	
12085	9	9	3	1	1	
12086	7	7	8	4	7	
12087	4	7	7	7	9	
12088	3	2	8	0	0	
12089	0	0	1	8	8	
12090	4	2	1	6	9	
12091	8	2	4	2	1	
12092	7	5	6	3	6	
12093	9	4	0	9	5	
12094	6	2	8	2	2	
12095	5	7	3	2	5	
12096	3	3	1	7	

12542	5	0	2	1	0	
12543	5	8	1	9	0	
12544	6	3	0	7	8	
12545	2	7	7	1	1	
12546	3	9	2	2	1	
12547	1	4	5	6	0	
12548	7	3	3	2	2	
12549	9	3	1	1	9	
12550	0	3	7	0	9	
12551	2	1	9	1	1	
12552	8	5	5	3	1	
12553	7	1	6	2	1	
12554	7	4	3	8	5	
12555	5	7	0	2	2	
12556	6	8	1	6	6	
12557	4	1	3	8	8	
12558	2	4	2	1	1	
12559	9	2	1	9	8	
12560	0	8	4	8	5	
12561	0	5	7	5	3	
12562	5	2	7	9	2	
12563	1	2	1	3	8	
12564	0	0	6	3	7	
12565	1	1	4	5	0	
12566	4	3	9	7	3	
12567	8	4	3	3	2	
12568	1	5	6	8	7	
12569	0	0	9	6	7	
12570	9	3	0	9	7	
12571	7	6	1	0	7	
12572	5	7	4	4	7	
12573	4	1	3	5	0	
12574	2	1	6	9	7	
12575	1	1	7	3	1	
12576	2	1	7	6	1	
12577	6	2	3	5	8	
12578	3	9	8	6	1	
12579	3	7	6	1	8	
12580	6	8	0	1	5	
12581	8	6	4	4	7	
12582	5	3	8	7	1	
12583	9	2	8	4	1	
12584	3	0	7	1	9	
12585	8	9	8	5	4	
12586	4	1	0	1	0	
12587	4	0	3	8	9	
12588	3	0	8	5	6	
12589	1	3	5	1	8	
12590	6	4	8	9	5	
12591	9	3	6	4	2	
12592	0	5	6	5	2	
12593	8	5	5	6	1	
12594	5	0	8	9	1	
12595	3	5	3	2	3	
12596	0	7	4	2	3	
12597	7	5	7	3	8	
12598	4	3	2	6	4	
12599	5	3	8	8	9	
12600	3	2	9	5	

13290	4	9	0	3	5	
13291	6	7	9	3	6	
13292	2	2	7	9	2	
13293	1	1	1	0	3	
13294	2	3	7	8	1	
13295	5	8	1	2	3	
13296	1	9	9	7	9	
13297	7	6	0	2	9	
13298	9	6	8	0	9	
13299	8	1	4	2	2	
13300	6	1	0	2	6	
13301	2	7	1	2	8	
13302	6	2	4	9	2	
13303	4	8	5	0	8	
13304	5	6	5	8	1	
13305	0	7	2	5	2	
13306	3	0	1	3	6	
13307	4	1	9	2	6	
13308	7	2	0	0	1	
13309	3	2	7	9	5	
13310	7	3	9	9	1	
13311	9	6	4	9	0	
13312	6	6	0	1	7	
13313	1	2	9	6	7	
13314	9	3	2	3	9	
13315	0	7	7	2	4	
13316	2	9	0	3	2	
13317	9	2	8	0	7	
13318	7	3	0	7	7	
13319	7	0	9	2	2	
13320	2	7	8	9	4	
13321	8	9	4	0	5	
13322	6	8	5	5	0	
13323	6	0	6	2	4	
13324	8	4	2	8	4	
13325	8	3	0	1	6	
13326	4	4	9	5	1	
13327	1	3	9	0	7	
13328	6	6	4	9	6	
13329	4	4	6	1	2	
13330	4	3	1	5	2	
13331	2	0	7	7	3	
13332	3	6	7	9	9	
13333	7	8	1	5	9	
13334	4	5	2	4	1	
13335	6	2	8	1	4	
13336	6	9	6	3	2	
13337	0	9	0	8	4	
13338	9	4	5	7	6	
13339	8	6	9	3	8	
13340	2	5	1	9	8	
13341	0	7	7	1	4	
13342	6	2	2	7	6	
13343	8	9	2	9	8	
13344	6	7	0	9	4	
13345	3	7	3	9	5	
13346	0	3	4	3	3	
13347	2	6	0	3	5	
13348	4	6	2	2	

# Sort each fold

In [9]:
for f in folds:
    f.sort()
print_folds(folds)

0	0	0	0	0	0	
1	0	0	0	0	0	
2	0	0	0	0	0	
3	0	0	0	0	0	
4	0	0	0	0	0	
5	0	0	0	0	0	
6	0	0	0	0	0	
7	0	0	0	0	0	
8	0	0	0	0	0	
9	0	0	0	0	0	
10	0	0	0	0	0	
11	0	0	0	0	0	
12	0	0	0	0	0	
13	0	0	0	0	0	
14	0	0	0	0	0	
15	0	0	0	0	0	
16	0	0	0	0	0	
17	0	0	0	0	0	
18	0	0	0	0	0	
19	0	0	0	0	0	
20	0	0	0	0	0	
21	0	0	0	0	0	
22	0	0	0	0	0	
23	0	0	0	0	0	
24	0	0	0	0	0	
25	0	0	0	0	0	
26	0	0	0	0	0	
27	0	0	0	0	0	
28	0	0	0	0	0	
29	0	0	0	0	0	
30	0	0	0	0	0	
31	0	0	0	0	0	
32	0	0	0	0	0	
33	0	0	0	0	0	
34	0	0	0	0	0	
35	0	0	0	0	0	
36	0	0	0	0	0	
37	0	0	0	0	0	
38	0	0	0	0	0	
39	0	0	0	0	0	
40	0	0	0	0	0	
41	0	0	0	0	0	
42	0	0	0	0	0	
43	0	0	0	0	0	
44	0	0	0	0	0	
45	0	0	0	0	0	
46	0	0	0	0	0	
47	0	0	0	0	0	
48	0	0	0	0	0	
49	0	0	0	0	0	
50	0	0	0	0	0	
51	0	0	0	0	0	
52	0	0	0	0	0	
53	0	0	0	0	0	
54	0	0	0	0	0	
55	0	0	0	0	0	
56	0	0	0	0	0	
57	0	0	0	0	0	
58	0	0	0	0	0	
59	0	0	0	0	0	
60	0	0	0	0	0	
61	0	0	0	0	0	
62	0	0	0	0	0	
63	0	0	0	0	0	
64	0	0	0	0	0	
65	0	0	0	0	0	
66	0	0	0	0	0	
67	0	0	0	0	0	
68	0	0	0	0	0	
69	0	0	0	0	0	
70	0	0	0	0	0	
71	0	0	0	0	0	
72

685	0	0	0	0	0	
686	0	0	0	0	0	
687	0	0	0	0	0	
688	0	0	0	0	0	
689	0	0	0	0	0	
690	0	0	0	0	0	
691	0	0	0	0	0	
692	0	0	0	0	0	
693	0	0	0	0	0	
694	0	0	0	0	0	
695	0	0	0	0	0	
696	0	0	0	0	0	
697	0	0	0	0	0	
698	0	0	0	0	0	
699	0	0	0	0	0	
700	0	0	0	0	0	
701	0	0	0	0	0	
702	0	0	0	0	0	
703	0	0	0	0	0	
704	0	0	0	0	0	
705	0	0	0	0	0	
706	0	0	0	0	0	
707	0	0	0	0	0	
708	0	0	0	0	0	
709	0	0	0	0	0	
710	0	0	0	0	0	
711	0	0	0	0	0	
712	0	0	0	0	0	
713	0	0	0	0	0	
714	0	0	0	0	0	
715	0	0	0	0	0	
716	0	0	0	0	0	
717	0	0	0	0	0	
718	0	0	0	0	0	
719	0	0	0	0	0	
720	0	0	0	0	0	
721	0	0	0	0	0	
722	0	0	0	0	0	
723	0	0	0	0	0	
724	0	0	0	0	0	
725	0	0	0	0	0	
726	0	0	0	0	0	
727	0	0	0	0	0	
728	0	0	0	0	0	
729	0	0	0	0	0	
730	0	0	0	0	0	
731	0	0	0	0	0	
732	0	0	0	0	0	
733	0	0	0	0	0	
734	0	0	0	0	0	
735	0	0	0	0	0	
736	0	0	0	0	0	
737	0	0	0	0	0	
738	0	0	0	0	0	
739	0	0	0	0	0	
740	0	0	0	0	0	
741	0	0	0	0	0	
742	0	0	0	0	0	
743	0	0	0	0	0	
744	0	0	0	0	0	
745	0	0	0	0	0	
746	0	0	0	0	0	
747	0	0	0	0	0	
748	0	0	0	0	0	
749	0	0	0	0	0	
750	0	0	0	0	0	
751	0	0	0	

1361	0	0	1	0	0	
1362	0	0	1	0	0	
1363	0	0	1	0	0	
1364	0	0	1	0	0	
1365	0	0	1	0	0	
1366	0	0	1	0	0	
1367	0	0	1	0	0	
1368	0	0	1	0	0	
1369	1	0	1	0	0	
1370	1	0	1	0	0	
1371	1	0	1	0	0	
1372	1	0	1	1	0	
1373	1	0	1	1	0	
1374	1	0	1	1	0	
1375	1	0	1	1	0	
1376	1	0	1	1	0	
1377	1	0	1	1	0	
1378	1	0	1	1	0	
1379	1	0	1	1	0	
1380	1	0	1	1	0	
1381	1	0	1	1	0	
1382	1	0	1	1	0	
1383	1	0	1	1	0	
1384	1	0	1	1	0	
1385	1	0	1	1	0	
1386	1	0	1	1	0	
1387	1	1	1	1	0	
1388	1	1	1	1	0	
1389	1	1	1	1	0	
1390	1	1	1	1	0	
1391	1	1	1	1	0	
1392	1	1	1	1	0	
1393	1	1	1	1	0	
1394	1	1	1	1	0	
1395	1	1	1	1	0	
1396	1	1	1	1	0	
1397	1	1	1	1	0	
1398	1	1	1	1	0	
1399	1	1	1	1	0	
1400	1	1	1	1	0	
1401	1	1	1	1	0	
1402	1	1	1	1	0	
1403	1	1	1	1	0	
1404	1	1	1	1	0	
1405	1	1	1	1	0	
1406	1	1	1	1	0	
1407	1	1	1	1	0	
1408	1	1	1	1	0	
1409	1	1	1	1	0	
1410	1	1	1	1	0	
1411	1	1	1	1	0	
1412	1	1	1	1	0	
1413	1	1	1	1	0	
1414	1	1	1	1	0	
1415	1	1	1	1	0	
1416	1	1	1	1	0	
1417	1	1	1	1	0	
1418	1	1	1	1	0	
1419	1	1	1	1	0	
1420	1	1	1	1	0	
1421	1	1	1	1	0	
1422	1	1	1	1	0	
1423	1	1

2060	1	1	1	1	1	
2061	1	1	1	1	1	
2062	1	1	1	1	1	
2063	1	1	1	1	1	
2064	1	1	1	1	1	
2065	1	1	1	1	1	
2066	1	1	1	1	1	
2067	1	1	1	1	1	
2068	1	1	1	1	1	
2069	1	1	1	1	1	
2070	1	1	1	1	1	
2071	1	1	1	1	1	
2072	1	1	1	1	1	
2073	1	1	1	1	1	
2074	1	1	1	1	1	
2075	1	1	1	1	1	
2076	1	1	1	1	1	
2077	1	1	1	1	1	
2078	1	1	1	1	1	
2079	1	1	1	1	1	
2080	1	1	1	1	1	
2081	1	1	1	1	1	
2082	1	1	1	1	1	
2083	1	1	1	1	1	
2084	1	1	1	1	1	
2085	1	1	1	1	1	
2086	1	1	1	1	1	
2087	1	1	1	1	1	
2088	1	1	1	1	1	
2089	1	1	1	1	1	
2090	1	1	1	1	1	
2091	1	1	1	1	1	
2092	1	1	1	1	1	
2093	1	1	1	1	1	
2094	1	1	1	1	1	
2095	1	1	1	1	1	
2096	1	1	1	1	1	
2097	1	1	1	1	1	
2098	1	1	1	1	1	
2099	1	1	1	1	1	
2100	1	1	1	1	1	
2101	1	1	1	1	1	
2102	1	1	1	1	1	
2103	1	1	1	1	1	
2104	1	1	1	1	1	
2105	1	1	1	1	1	
2106	1	1	1	1	1	
2107	1	1	1	1	1	
2108	1	1	1	1	1	
2109	1	1	1	1	1	
2110	1	1	1	1	1	
2111	1	1	1	1	1	
2112	1	1	1	1	1	
2113	1	1	1	1	1	
2114	1	1	1	1	1	
2115	1	1	1	1	1	
2116	1	1	1	1	1	
2117	1	1	1	1	1	
2118	1	1	1	1	1	
2119	1	1	1	1	1	
2120	1	1	1	1	1	
2121	1	1	1	1	1	
2122	1	1

2596	1	1	1	1	1	
2597	1	1	1	1	1	
2598	1	1	1	1	1	
2599	1	1	1	1	1	
2600	1	1	1	1	1	
2601	1	1	1	1	1	
2602	1	1	1	1	1	
2603	1	1	1	1	1	
2604	1	1	1	1	1	
2605	1	1	1	1	1	
2606	1	1	1	1	1	
2607	1	1	1	1	1	
2608	1	1	1	1	1	
2609	1	1	1	1	1	
2610	1	1	1	1	1	
2611	1	1	1	1	1	
2612	1	1	1	1	1	
2613	1	1	1	1	1	
2614	1	1	1	1	1	
2615	1	1	1	1	1	
2616	1	1	1	1	1	
2617	1	1	1	1	1	
2618	1	1	1	1	1	
2619	1	1	1	1	1	
2620	1	1	1	1	1	
2621	1	1	1	1	1	
2622	1	1	1	1	1	
2623	1	1	1	1	1	
2624	1	1	1	1	1	
2625	1	1	1	1	1	
2626	1	1	1	1	1	
2627	1	1	1	1	1	
2628	1	1	1	1	1	
2629	1	1	1	1	1	
2630	1	1	1	1	1	
2631	1	1	1	1	1	
2632	1	1	1	1	1	
2633	1	1	1	1	1	
2634	1	1	1	1	1	
2635	1	1	1	1	1	
2636	1	1	1	1	1	
2637	1	1	1	1	1	
2638	1	1	1	1	1	
2639	1	1	1	1	1	
2640	1	1	1	1	1	
2641	1	1	1	1	1	
2642	1	1	1	1	1	
2643	1	1	1	1	1	
2644	1	1	1	1	1	
2645	1	1	1	1	1	
2646	1	1	1	1	1	
2647	1	1	1	1	1	
2648	1	1	1	1	1	
2649	1	1	1	1	1	
2650	1	1	1	1	1	
2651	1	1	1	1	1	
2652	1	1	1	1	1	
2653	1	1	1	1	1	
2654	1	1	1	1	1	
2655	1	1	1	1	1	
2656	1	1	1	1	1	
2657	1	1	1	1	1	
2658	1	1

...

11656	8	8	8	8	8	
11657	8	8	8	8	8	
11658	8	8	8	8	8	
11659	8	8	8	8	8	
11660	8	8	8	8	8	
11661	8	8	8	8	8	
11662	8	8	8	8	8	
11663	8	8	8	8	8	
11664	8	8	8	8	8	
11665	8	8	8	8	8	
11666	8	8	8	8	8	
11667	8	8	8	8	8	
11668	8	8	8	8	8	
11669	8	8	8	8	8	
11670	8	8	8	8	8	
11671	8	8	8	8	8	
11672	8	8	8	8	8	
11673	8	8	8	8	8	
11674	8	8	8	8	8	
11675	8	8	8	8	8	
11676	8	8	8	8	8	
11677	8	8	8	8	8	
11678	8	8	8	8	8	
11679	8	8	8	8	8	
11680	8	8	8	8	8	
11681	8	8	8	8	8	
11682	8	8	8	8	8	
11683	8	8	8	8	8	
11684	8	8	8	8	8	
11685	8	8	8	8	8	
11686	8	8	8	8	8	
11687	8	8	8	8	8	
11688	8	8	8	8	8	
11689	8	8	8	8	8	
11690	8	8	8	8	8	
11691	8	8	8	8	8	
11692	8	8	8	8	8	
11693	8	8	8	8	8	
11694	8	8	8	8	8	
11695	8	8	8	8	8	
11696	8	8	8	8	8	
11697	8	8	8	8	8	
11698	8	8	8	8	8	
11699	8	8	8	8	8	
11700	8	8	8	8	8	
11701	8	8	8	8	8	
11702	8	8	8	8	8	
11703	8	8	8	8	8	
11704	8	8	8	8	8	
11705	8	8	8	8	8	
11706	8	8	8	8	8	
11707	8	8	8	8	8	
11708	8	8	8	8	8	
11709	8	8	8	8	8	
11710	8	8	8	8	8	
11711	8	8	8	8	8	
11712	8	8	8	8	8	
11713	8	8	8	8	8	
11714	8	8	8	8	

12175	8	8	8	8	8	
12176	8	8	8	8	8	
12177	8	8	8	8	8	
12178	8	8	8	8	8	
12179	8	8	8	8	8	
12180	8	8	8	8	8	
12181	8	8	8	8	8	
12182	8	8	8	8	8	
12183	8	8	8	8	8	
12184	8	8	8	8	8	
12185	8	8	8	8	8	
12186	8	8	8	8	8	
12187	8	8	8	8	8	
12188	8	8	8	8	8	
12189	8	8	8	8	8	
12190	8	8	8	8	8	
12191	8	8	8	8	8	
12192	8	8	8	8	8	
12193	8	8	8	8	8	
12194	8	8	8	8	8	
12195	8	8	8	8	8	
12196	8	8	8	8	8	
12197	8	8	8	8	8	
12198	8	8	8	8	8	
12199	8	8	8	8	8	
12200	8	8	8	8	8	
12201	8	8	8	8	8	
12202	8	8	8	8	8	
12203	8	8	8	8	8	
12204	8	8	8	8	8	
12205	8	8	8	8	8	
12206	8	8	8	8	8	
12207	8	8	8	8	8	
12208	8	8	8	8	8	
12209	8	8	8	8	8	
12210	8	8	8	8	8	
12211	8	8	8	8	8	
12212	8	8	8	8	8	
12213	8	8	8	8	8	
12214	8	8	8	8	8	
12215	8	8	8	8	8	
12216	8	8	8	8	8	
12217	8	8	8	8	8	
12218	8	8	8	8	8	
12219	8	8	8	8	8	
12220	8	8	8	8	8	
12221	8	8	8	8	8	
12222	8	8	8	8	8	
12223	8	8	8	8	8	
12224	8	8	8	8	8	
12225	8	8	8	8	8	
12226	8	8	8	8	8	
12227	8	8	8	8	8	
12228	8	8	8	8	8	
12229	8	8	8	8	8	
12230	8	8	8	8	8	
12231	8	8	8	8	8	
12232	8	8	8	8	8	
12233	8	8	8	8	

12679	9	9	9	9	9	
12680	9	9	9	9	9	
12681	9	9	9	9	9	
12682	9	9	9	9	9	
12683	9	9	9	9	9	
12684	9	9	9	9	9	
12685	9	9	9	9	9	
12686	9	9	9	9	9	
12687	9	9	9	9	9	
12688	9	9	9	9	9	
12689	9	9	9	9	9	
12690	9	9	9	9	9	
12691	9	9	9	9	9	
12692	9	9	9	9	9	
12693	9	9	9	9	9	
12694	9	9	9	9	9	
12695	9	9	9	9	9	
12696	9	9	9	9	9	
12697	9	9	9	9	9	
12698	9	9	9	9	9	
12699	9	9	9	9	9	
12700	9	9	9	9	9	
12701	9	9	9	9	9	
12702	9	9	9	9	9	
12703	9	9	9	9	9	
12704	9	9	9	9	9	
12705	9	9	9	9	9	
12706	9	9	9	9	9	
12707	9	9	9	9	9	
12708	9	9	9	9	9	
12709	9	9	9	9	9	
12710	9	9	9	9	9	
12711	9	9	9	9	9	
12712	9	9	9	9	9	
12713	9	9	9	9	9	
12714	9	9	9	9	9	
12715	9	9	9	9	9	
12716	9	9	9	9	9	
12717	9	9	9	9	9	
12718	9	9	9	9	9	
12719	9	9	9	9	9	
12720	9	9	9	9	9	
12721	9	9	9	9	9	
12722	9	9	9	9	9	
12723	9	9	9	9	9	
12724	9	9	9	9	9	
12725	9	9	9	9	9	
12726	9	9	9	9	9	
12727	9	9	9	9	9	
12728	9	9	9	9	9	
12729	9	9	9	9	9	
12730	9	9	9	9	9	
12731	9	9	9	9	9	
12732	9	9	9	9	9	
12733	9	9	9	9	9	
12734	9	9	9	9	9	
12735	9	9	9	9	9	
12736	9	9	9	9	9	
12737	9	9	9	9	

13184	9	9	9	9	9	
13185	9	9	9	9	9	
13186	9	9	9	9	9	
13187	9	9	9	9	9	
13188	9	9	9	9	9	
13189	9	9	9	9	9	
13190	9	9	9	9	9	
13191	9	9	9	9	9	
13192	9	9	9	9	9	
13193	9	9	9	9	9	
13194	9	9	9	9	9	
13195	9	9	9	9	9	
13196	9	9	9	9	9	
13197	9	9	9	9	9	
13198	9	9	9	9	9	
13199	9	9	9	9	9	
13200	9	9	9	9	9	
13201	9	9	9	9	9	
13202	9	9	9	9	9	
13203	9	9	9	9	9	
13204	9	9	9	9	9	
13205	9	9	9	9	9	
13206	9	9	9	9	9	
13207	9	9	9	9	9	
13208	9	9	9	9	9	
13209	9	9	9	9	9	
13210	9	9	9	9	9	
13211	9	9	9	9	9	
13212	9	9	9	9	9	
13213	9	9	9	9	9	
13214	9	9	9	9	9	
13215	9	9	9	9	9	
13216	9	9	9	9	9	
13217	9	9	9	9	9	
13218	9	9	9	9	9	
13219	9	9	9	9	9	
13220	9	9	9	9	9	
13221	9	9	9	9	9	
13222	9	9	9	9	9	
13223	9	9	9	9	9	
13224	9	9	9	9	9	
13225	9	9	9	9	9	
13226	9	9	9	9	9	
13227	9	9	9	9	9	
13228	9	9	9	9	9	
13229	9	9	9	9	9	
13230	9	9	9	9	9	
13231	9	9	9	9	9	
13232	9	9	9	9	9	
13233	9	9	9	9	9	
13234	9	9	9	9	9	
13235	9	9	9	9	9	
13236	9	9	9	9	9	
13237	9	9	9	9	9	
13238	9	9	9	9	9	
13239	9	9	9	9	9	
13240	9	9	9	9	9	
13241	9	9	9	9	9	
13242	9	9	9	9	

13914	9	9	9	9	9	
13915	9	9	9	9	9	
13916	9	9	9	9	9	
13917	9	9	9	9	9	
13918	9	9	9	9	9	
13919	9	9	9	9	9	
13920	9	9	9	9	9	
13921	9	9	9	9	9	
13922	9	9	9	9	9	
13923	9	9	9	9	9	
13924	9	9	9	9	9	
13925	9	9	9	9	9	
13926	9	9	9	9	9	
13927	9	9	9	9	9	
13928	9	9	9	9	9	
13929	9	9	9	9	9	
13930	9	9	9	9	9	
13931	9	9	9	9	9	
13932	9	9	9	9	9	
13933	9	9	9	9	9	
13934	9	9	9	9	9	
13935	9	9	9	9	9	
13936	9	9	9	9	9	
13937	9	9	9	9	9	
13938	9	9	9	9	9	
13939	9	9	9	9	9	
13940	9	9	9	9	9	
13941	9	9	9	9	9	
13942	9	9	9	9	9	
13943	9	9	9	9	9	
13944	9	9	9	9	9	
13945	9	9	9	9	9	
13946	9	9	9	9	9	
13947	9	9	9	9	9	
13948	9	9	9	9	9	
13949	9	9	9	9	9	
13950	9	9	9	9	9	
13951	9	9	9	9	9	
13952	9	9	9	9	9	
13953	9	9	9	9	9	
13954	9	9	9	9	9	
13955	9	9	9	9	9	
13956	9	9	9	9	9	
13957	9	9	9	9	9	
13958	9	9	9	9	9	
13959	9	9	9	9	9	
13960	9	9	9	9	9	
13961	9	9	9	9	9	
13962	9	9	9	9	9	
13963	9	9	9	9	9	
13964	9	9	9	9	9	
13965	9	9	9	9	9	
13966	9	9	9	9	9	
13967	9	9	9	9	9	
13968	9	9	9	9	9	
13969	9	9	9	9	9	
13970	9	9	9	9	9	
13971	9	9	9	9	9	
13972	9	9	9	9	

In [10]:
from itertools import groupby
from operator import itemgetter

def data_fold(fold):
    groups = groupby(fold, itemgetter(0))
    for (key, data) in groups:
        key = int.from_bytes(key, byteorder='big')
        n_of_digit = len(list(data))
        yield (key, n_of_digit)
        
print("The number of sample for each digit for each fold:")
print_gens = [data_fold(f) for f in folds]
for key_and_digit in zip(*print_gens):
    dup_key, n_per_group = list(zip(*key_and_digit))
    key = dup_key[0]
    print(key, end='\t')
    print(n_per_group, end='\t')
    print()
    

The number of sample for each digit for each fold:
0	(1369, 1387, 1343, 1372, 1432)	
1	(1618, 1601, 1583, 1529, 1546)	
2	(1411, 1350, 1380, 1397, 1452)	
3	(1352, 1482, 1461, 1511, 1335)	
4	(1350, 1342, 1374, 1346, 1412)	
5	(1318, 1278, 1241, 1248, 1228)	
6	(1366, 1396, 1416, 1403, 1295)	
7	(1450, 1470, 1470, 1456, 1447)	
8	(1352, 1341, 1352, 1383, 1397)	
9	(1414, 1353, 1380, 1355, 1456)	


# Group into 200 shards of size 70 for each fold

In [11]:
SHARD_SIZE = 70
def get_shards(fold):
    shards = []
    fold_len = len(fold)
    n_shards = fold_len // SHARD_SIZE
    assert(fold_len % SHARD_SIZE == 0)
    for n in range(n_shards):
        shards.append(fold[:SHARD_SIZE])
        fold = fold[SHARD_SIZE:]
    return shards
        
shards_folds = [get_shards(f) for f in folds]
print("shards_folds dimensions:")
print(len(shards_folds))
print(len(shards_folds[-1]))
print(len(shards_folds[-1][-1]))

shards_folds dimensions:
5
200
70


# Suffle each fold with the same seed

In [12]:
def suffle_them_same(lists):
    for li in lists:
        # set seed
        seed(SEED)
        # shuffle
        shuffle(li)
        

suffle_them_same(shards_folds)

# Write data to files

In [13]:
import os

directory = DATA_DIR.parent / 'mnist_noniid_cv'
try:
    os.mkdir(directory)
except FileExistsError:
    print("A folder already exists:", directory)

for i in range(NUM_FOLDS):
    folder_name = directory / ('fold' + str(1+i))
    try:
        os.mkdir(folder_name)
    except FileExistsError:
        print("A folder already exists:", folder_name)

A folder already exists: ../src/data/mnist_noniid_cv


In [14]:
os.listdir(directory)

['fold2', 'fold4', 'fold3', 'fold1', 'fold5']

In [15]:
def write_non_iid(shards, output_dir):
    '''
    shards: a list of fragments of the sorted list of input/label pairs
    output_dir: where to put the resulting files

    This function writes two shards per car.
    Same thing as the FedAvg papper does.
    '''
    n_shards = len(shards)
    n_k = len(shards[0])*2
    for i2 in range(0, n_shards, 2):
        car_i = i2 // 2 + 1

        lbls_file_name = output_dir / f"car{car_i}-labels.byte"
        imgs_file_name = output_dir / f"car{car_i}-images.byte"
        with open(lbls_file_name, 'wb') as lbl_file, \
                open(imgs_file_name, 'wb') as img_file:

            lbl_magic_nr = b'\x00\x00\x08\x01' # ubyte, 1-dim
            n_lbl = (n_k).to_bytes(4, byteorder='big')
            lbl_header = lbl_magic_nr + n_lbl
            lbl_file.write(lbl_header)

            img_magic_nr = b'\x00\x00\x08\x03' # ubyte, 3-dim
            n_imgs = (n_k).to_bytes(4, byteorder='big')
            n_rows = (28).to_bytes(4, byteorder='big')
            n_cols = (28).to_bytes(4, byteorder='big')
            img_header = img_magic_nr + n_imgs + n_rows + n_cols
            img_file.write(img_header)

            for shard in (shards[i2], shards[i2+1]):
                for (lbl, img) in shard:
                    lbl_file.write(lbl)
                    img_file.write(img)

In [16]:
folders = os.listdir(directory)
folders.sort()
for shards, dire in zip(shards_folds, folders):
    output_dir = directory / dire
    print(output_dir)
    write_non_iid(shards, output_dir)

../src/data/mnist_noniid_cv/fold1
../src/data/mnist_noniid_cv/fold2
../src/data/mnist_noniid_cv/fold3
../src/data/mnist_noniid_cv/fold4
../src/data/mnist_noniid_cv/fold5


# Read how many digit each client has

In [17]:
import numpy
import struct

def read_mnist_data(fname_img, fname_lbl):
    '''
    Read MNIST data from a byte file.

    Return: tuple of inputs and labels (numpy)
    '''
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = numpy.fromfile(flbl, dtype=numpy.int8)
        if len(lbl) != num:
            print('Header mismatch. #labels != header number')

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = numpy.fromfile(fimg, dtype=numpy.uint8).reshape(num, rows, cols)

    return (img, lbl)

In [18]:
import os

directory = DATA_DIR.parent / 'mnist_noniid_cv'

def read_car_data(folder_names, input_dir, car_i):
    '''
    Read data for one car
    '''
    x_return = []
    y_return = []

    for fold_name in folder_names:
        # Read file
        input_directory = input_dir / fold_name
        label_file = input_directory / f"car{car_i}-labels.byte"
        image_file = input_directory / f"car{car_i}-images.byte"

        x, y = read_mnist_data(image_file, label_file)

        # accumulate/store chosen
        x_return.append(x)
        y_return.append(y)

    # return
    return (numpy.concatenate(x_return), numpy.concatenate(y_return))
    
folders = os.listdir(directory)
folders.remove('fold5')

list_of_pairs = [read_car_data(folders, directory, car_i) for car_i in range(1, 101)]

In [19]:
from itertools import groupby

freq = dir()
for i in range(10):
    freq[i] = 0

for i,(_, car_lbls) in enumerate(list_of_pairs):
    groups = groupby(numpy.sort(car_lbls))
    keys = [key for (key, data) in groups]
    print(i+1, keys)
    for num in keys:
        freq[num] += 1
            
print("Frequency")
for i in range(10):
    print(i,":", freq[i])

1 [2, 7]
2 [1, 7]
3 [4, 8]
4 [1]
5 [0, 7]
6 [1, 2]
7 [0, 1]
8 [3, 4, 8, 9]
9 [2, 3]
10 [1, 6]
11 [2, 7]
12 [0, 5]
13 [5, 6, 8]
14 [0, 9]
15 [6, 7]
16 [5, 7, 8]
17 [3]
18 [1, 2, 5]
19 [2, 5]
20 [5, 9]
21 [6, 8]
22 [5, 6]
23 [1, 4]
24 [0, 6]
25 [7, 9]
26 [0, 4, 5]
27 [7, 9]
28 [1, 3, 4]
29 [8, 9]
30 [3, 6]
31 [0, 6]
32 [3, 4]
33 [2, 6, 7]
34 [3, 5]
35 [2, 7]
36 [0, 1]
37 [6, 7]
38 [1, 9]
39 [5, 6]
40 [1, 4]
41 [1, 2, 3]
42 [7, 8]
43 [4, 8]
44 [2, 3]
45 [3, 6]
46 [4, 9]
47 [1, 4]
48 [0, 3]
49 [0, 2]
50 [6, 7]
51 [3, 8]
52 [2, 9]
53 [3]
54 [2, 6]
55 [2, 7]
56 [5, 9]
57 [1, 2]
58 [1, 9]
59 [4, 6]
60 [1, 9]
61 [1, 4, 5]
62 [3, 9]
63 [3, 6]
64 [2, 8]
65 [6, 8]
66 [7]
67 [2, 7]
68 [4, 5]
69 [7, 8]
70 [0, 8]
71 [5, 6, 9]
72 [1, 2]
73 [6, 8]
74 [0, 5]
75 [5, 7]
76 [7, 9]
77 [4, 9]
78 [1, 4]
79 [0, 3]
80 [1, 8]
81 [4, 5]
82 [1, 4]
83 [5]
84 [4, 7]
85 [0, 9]
86 [4, 6]
87 [0, 8]
88 [7, 8]
89 [4, 9]
90 [2, 3]
91 [2, 3]
92 [5, 8]
93 [1, 4]
94 [0, 8, 9]
95 [0, 3]
96 [0]
97 [7, 8, 9]
98 [3, 6]
99 [2, 8

## Any underrepresented numbers?

In [20]:
import os
from itertools import groupby

directory = '../src/data/mnist_noniid_cv/'

def read_car_data(folder_names, input_dir, car_i):
    '''
    Read data for one car
    '''
    x_return = []
    y_return = []

    for fold_name in folder_names:
        # Read file
        input_directory = input_dir + fold_name
        label_file = "{}/car{}-labels.byte".format(input_directory, car_i)
        image_file = "{}/car{}-images.byte".format(input_directory, car_i)

        x, y = read_mnist_data(image_file, label_file)

        # accumulate/store chosen
        x_return.append(x)
        y_return.append(y)

    # return
    return (numpy.concatenate(x_return), numpy.concatenate(y_return))
    
folders = os.listdir(directory)

for fold in folders:
    list_of_pairs = [read_car_data([fold], directory, car_i) for car_i in range(1, 101)]

    xs, ys = zip(*list_of_pairs)
    imgs, lbs = numpy.concatenate(xs), numpy.concatenate(ys)

    groups = groupby(numpy.sort(lbs))
    lens = [(key, len(list(data))) for (key, data) in groups]
    print(fold,lens)

fold2 [(0, 1387), (1, 1601), (2, 1350), (3, 1482), (4, 1342), (5, 1278), (6, 1396), (7, 1470), (8, 1341), (9, 1353)]
fold4 [(0, 1372), (1, 1529), (2, 1397), (3, 1511), (4, 1346), (5, 1248), (6, 1403), (7, 1456), (8, 1383), (9, 1355)]
fold3 [(0, 1343), (1, 1583), (2, 1380), (3, 1461), (4, 1374), (5, 1241), (6, 1416), (7, 1470), (8, 1352), (9, 1380)]
fold1 [(0, 1369), (1, 1618), (2, 1411), (3, 1352), (4, 1350), (5, 1318), (6, 1366), (7, 1450), (8, 1352), (9, 1414)]
fold5 [(0, 1432), (1, 1546), (2, 1452), (3, 1335), (4, 1412), (5, 1228), (6, 1295), (7, 1447), (8, 1397), (9, 1456)]
